In [1]:
%doctest_mode

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

# Import the linear regression class
from sklearn.linear_model import LinearRegression

# Sklearn also has a helper that makes it easy to do cross validation
from sklearn.cross_validation import KFold

# train test split made easy
from sklearn.cross_validation import train_test_split

#statistics and Integration
from scipy import stats, integrate


# Import preprocessing class
from sklearn import preprocessing



Exception reporting mode: Plain
Doctest mode is: ON


In [2]:
# read in the cvs
cf_pd = pd.read_csv("cf.csv", verbose=False)

# get the data for predictions
cf_in = pd.read_csv("singleScript.csv", verbose=False)

#create the Profit feature where Profit = 1 if BOX / BUDGET >= 1 and 0 otherwise
cf_pd["Profit"] = cf_pd["BOX"] / cf_pd["BUDGET"]

cf_pd['Profit'].loc[cf_pd['Profit'] >= 1.] = 1
cf_pd['Profit'].loc[cf_pd['Profit'] < 1.] = 0

#80/20 split into train and test
cf_pd_train, cf_pd_test = train_test_split(cf_pd, test_size=0.2)

/home/swissj74/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [3]:
#create a dataframe to hold the results
#contains the model name, the features used, movie name, and predicted box
#results = pd.DataFrame(index = ['lr_model', 'rr_model', 'rr_cv_model', 'lasso_cv_model', 'rr_cv_lasso_model'
#                                , 'elastic_net_model', 'elastic_net_model']
#                                , columns=['Features', 'MovieName', 'PredictedBox'])

In [4]:
my_features = ['BUDGET', 'scrCnt', 'GENRE_DRA', 'GENRE_ROM', 'GENRE_THR', 'GENRE_COM', 'GENRE_HOR', 'GENRE_SCI'
               , 'GENRE_ACT', 'GENRE_FAM', 'NTITLE', 'YEAR', 'avgClrPrem', 'avgImpPrem', 'avgFamSet', 'avgEarExp'
               , 'avgCoAvoid', 'avgIntCon', 'avgSurp', 'avgAntici', 'avgFlhback', 'avgClrMot', 'avgMulDim'
               , 'avgHeroW', 'avgStrNem', 'avgSymHero', 'avgLogic', 'avgCharGrow', 'avgImp', 'avgMulConf'
               , 'avgIntensity', 'avgBuild', 'avgLockIn', 'avgResolut', 'avgBelieve', 'avgSurpEnd', 'NSCENE'
               , 'INTPREC']

In [5]:
# Normalizing the data
scaler = preprocessing.StandardScaler().fit(cf_pd_train[my_features])
cf_pd_train.loc[:, my_features] = scaler.transform(cf_pd_train[my_features])
cf_pd_test.loc[:, my_features] = scaler.transform(cf_pd_test[my_features])
cf_in.loc[:, my_features] = scaler.transform(cf_in[my_features])

/home/swissj74/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [6]:
from sklearn import linear_model
lr_model = linear_model.LinearRegression()
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)
lr_model.fit(cf_pd_train[my_features], cf_pd_train["BOX"])


# The coefficients
print('Coefficients: \n', lr_model.coef_)
# The mean square error
print("Residual sum of squares: %.2f"
      % np.mean((lr_model.predict(cf_pd_test[my_features]) - cf_pd_test["BOX"]) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % lr_model.score(cf_pd_test[my_features], cf_pd_test["BOX"]))
# Predict with all features
print('Predicted Box Office: %.2f' % lr_model.predict(cf_in[my_features]))

#results.PredictedBox.loc['lr_model'] = lr_model.predict(cf_in[my_features])
#results.MovieName.loc['lr_model'] = cf_in.TITLE.loc[0]
#results.Features.loc['lr_model'] = my_features

Coefficients: 
 [ 37.43563578  18.81930685 -10.82414368  -4.00940937  -8.91094071
  -6.27864241 -12.64023581  -3.86843895  -2.20658594  12.10125048
  19.34251427  -4.65353041  -2.05831243 -12.79455824  17.52012817
   0.91976729  -6.77747734   3.14823717   9.76175469  -4.95787714
   1.67566109  -0.80109124   2.63308375   9.6695825   -1.81167212
   6.49593475   9.00971951   4.3377218   -4.70509557   6.08743735
   3.54428934   0.06100503  12.24067728  -7.77444317  -9.17207415
  -3.93576902  10.08626861   0.5303532 ]
Residual sum of squares: 4353.27
Variance score: 0.26
Predicted Box Office: 230.75


In [13]:
print('Model\tResidual sum of Squares\tVariance\tPredicted Box Office')
# The coefficients
# print('Coefficients: \n', lr_model.coef_)
# The mean square error
rss = np.mean((lr_model.predict(cf_pd_test[my_features]) - cf_pd_test["BOX"]) ** 2)
# Explained variance score: 1 is perfect prediction
variance = lr_model.score(cf_pd_test[my_features], cf_pd_test["BOX"])
# Predict with all features
Predicted_Box_Office = lr_model.predict(cf_in[my_features])[0]
model = 'lr model'
#results.PredictedBox.loc['lr_model'] = lr_model.predict(cf_in[my_features])
#results.MovieName.loc['lr_model'] = cf_in.TITLE.loc[0]
#results.Features.loc['lr_model'] = my_features

print(model, '\t', rss, '\t', variance , '\t', Predicted_Box_Office)

Model	Residual sum of Squares	Variance	Predicted Box Office
lr model 	 4353.266410821524 	 0.256923861868 	 230.74586022


[ 230.74586022]


In [7]:
for rr_alpha in [0.1, 1, 10]:
    rr_model = linear_model.Ridge (alpha = rr_alpha)
    rr_model.fit(cf_pd_train[my_features], cf_pd_train["BOX"])

    # The alpha
    print('alpha: ', str(rr_alpha))
    # The coefficients
    print('Coefficients: \n', rr_model.coef_)
    # The mean square error
    print("Residual sum of squares: %.2f"
          % np.mean((rr_model.predict(cf_pd_test[my_features]) - cf_pd_test["BOX"]) ** 2))
    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % rr_model.score(cf_pd_test[my_features], cf_pd_test["BOX"]))
    # Predict with all features
    print('Predicted Box Office: %.2f' % rr_model.predict(cf_in[my_features]))
    


rr_cv_model = linear_model.RidgeCV()
rr_cv_model.fit(cf_pd_train[my_features], cf_pd_train["BOX"])

# The alpha
print('alpha: ', rr_cv_model.alpha_)
# The coefficients
print('Coefficients: \n', rr_cv_model.coef_)
# The mean square error
print("Residual sum of squares: %.2f"
      % np.mean((rr_cv_model.predict(cf_pd_test[my_features]) - cf_pd_test["BOX"]) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % rr_cv_model.score(cf_pd_test[my_features], cf_pd_test["BOX"]))
# Predict with all features
print('Predicted Box Office: %.2f' % rr_cv_model.predict(cf_in[my_features]))

alpha:  0.1
Coefficients: 
 [ 33.78486316  21.45526177  -9.09497951  -7.1868801   -7.29387694
  -2.9090056  -15.29124561   0.79536145  -1.82614006  15.88092665
  15.90023898 -10.52375116   1.43486902  -9.9592065   12.85603777
   4.43831812  -4.21291622  -1.99575216  10.91470169   1.05910122
  -2.48716171  -7.35287539   3.82518301   8.79706805   3.32831859
   3.52115128  11.55815353   6.74506024  -0.37551963   1.05402391
  -0.18654689  -1.68482271   8.86203935  -8.09573028  -3.35948173
  -3.4680262    9.67866603   0.17021708]
Residual sum of squares: 2921.29
Variance score: 0.46
Predicted Box Office: -82.20
alpha:  1
Coefficients: 
 [  3.34931575e+01   2.14523951e+01  -8.89786549e+00  -7.03557373e+00
  -7.19304169e+00  -2.75068543e+00  -1.49452287e+01   7.62030433e-01
  -1.61741724e+00   1.58540690e+01   1.56247580e+01  -1.02842593e+01
   1.48931471e+00  -9.77824061e+00   1.26945480e+01   4.37111080e+00
  -4.10880086e+00  -1.83055872e+00   1.07593405e+01   1.05795639e+00
  -2.49195079e+

In [ ]:
lasso_cv_model = linear_model.LassoCV()
lasso_cv_model.fit(cf_pd_train[my_features], cf_pd_train["BOX"])

# The alpha
print('alpha: ', lasso_cv_model.alpha_)
# The coefficients
print('Coefficients: \n', lasso_cv_model.coef_)
# The mean square error
print("Residual sum of squares: %.2f"
      % np.mean((lasso_cv_model.predict(cf_pd_test[my_features]) - cf_pd_test["BOX"]) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % lasso_cv_model.score(cf_pd_test[my_features], cf_pd_test["BOX"]))
# Predict with all features
print('Predicted Box Office: %.2f' % lasso_cv_model.predict(cf_in[my_features]))


In [ ]:
my_features_lasso = ['BUDGET', 'scrCnt', 'GENRE_THR', 'GENRE_COM', 'GENRE_HOR', 'GENRE_FAM', 'NTITLE'
                     , 'avgImpPrem', 'avgEarExp', 'avgLockIn', 'avgResolut', 'NSCENE']

rr_cv_lasso_model = linear_model.RidgeCV()
rr_cv_lasso_model.fit(cf_pd_train[my_features_lasso], cf_pd_train["BOX"])

# The alpha
print('alpha: ', rr_cv_lasso_model.alpha_)
# The coefficients
print('Coefficients: \n', rr_cv_lasso_model.coef_)
# The mean square error
print("Residual sum of squares: %.2f"
      % np.mean((rr_cv_lasso_model.predict(cf_pd_test[my_features_lasso]) - cf_pd_test["BOX"]) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % rr_cv_lasso_model.score(cf_pd_test[my_features_lasso], cf_pd_test["BOX"]))
# Predict with all features
print('Predicted Box Office: %.2f' % rr_cv_lasso_model.predict(cf_in[my_features_lasso]))


In [ ]:
elastic_net_model = linear_model.ElasticNetCV()
elastic_net_model.fit(cf_pd_train[my_features], cf_pd_train["BOX"])

# The alpha
print('alpha: ', elastic_net_model.alpha_)
# The l1 ration
print('l1 ratio: ', elastic_net_model.l1_ratio_)
# The coefficients
print('Coefficients: \n', elastic_net_model.coef_)
# The mean square error
print("Residual sum of squares: %.2f"
      % np.mean((elastic_net_model.predict(cf_pd_test[my_features]) - cf_pd_test["BOX"]) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % elastic_net_model.score(cf_pd_test[my_features], cf_pd_test["BOX"]))
# Predict with all features
print('Predicted Box Office: %.2f' % elastic_net_model.predict(cf_in[my_features]))


In [ ]:
elastic_net_model = linear_model.ElasticNetCV()
elastic_net_model.fit(cf_pd_train[my_features_lasso], cf_pd_train["BOX"])

# The alpha
print('alpha: ', elastic_net_model.alpha_)
# The l1 ration
print('l1 ratio: ', elastic_net_model.l1_ratio_)
# The coefficients
print('Coefficients: \n', elastic_net_model.coef_)
# The mean square error
print("Residual sum of squares: %.2f"
      % np.mean((elastic_net_model.predict(cf_pd_test[my_features_lasso]) - cf_pd_test["BOX"]) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % elastic_net_model.score(cf_pd_test[my_features_lasso], cf_pd_test["BOX"]))
# Predict with all features
print('Predicted Box Office: %.2f' % elastic_net_model.predict(cf_in[my_features_lasso]))


In [ ]:
my_features_small = ['BUDGET', 'scrCnt', 'NSCENE']

from sklearn import linear_model
lr_model_small = linear_model.LinearRegression()
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)
lr_model_small.fit(cf_pd_train[my_features_small], cf_pd_train["BOX"])


# The coefficients
print('Coefficients: \n', lr_model_small.coef_)
# The mean square error
print("Residual sum of squares: %.2f"
      % np.mean((lr_model_small.predict(cf_pd_test[my_features_small]) - cf_pd_test["BOX"]) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % lr_model_small.score(cf_pd_test[my_features_small], cf_pd_test["BOX"]))
# Predict with all features
print('Predicted Box Office: %.2f' % lr_model_small.predict(cf_in[my_features_small]))

In [ ]:
lasso_lars_model = linear_model.LassoLarsCV(fit_intercept=True)
lasso_lars_model.fit(cf_pd_train[my_features], cf_pd_train["BOX"])

# The alpha
print('alpha: ', lasso_lars_model.alpha_)
# The coefficients
print('Coefficients: \n', lasso_lars_model.coef_)
# The mean square error
print("Residual sum of squares: %.2f"
      % np.mean((lasso_lars_model.predict(cf_pd_test[my_features]) - cf_pd_test["BOX"]) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % lasso_lars_model.score(cf_pd_test[my_features], cf_pd_test["BOX"]))
# Predict with all features
print('Predicted Box Office: %.2f' % lasso_lars_model.predict(cf_in[my_features]))

In [ ]:
my_lasso_lars_features = ['BUDGET', 'scrCnt', 'GENRE_FAM', 'NTITLE']

from sklearn import linear_model
lr_model_small_lasso_lars = linear_model.LinearRegression()
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)
lr_model_small_lasso_lars.fit(cf_pd_train[my_lasso_lars_features], cf_pd_train["BOX"])


# The coefficients
print('Coefficients: \n', lr_model_small_lasso_lars.coef_)
# The mean square error
print("Residual sum of squares: %.2f"
      % np.mean((lr_model_small_lasso_lars.predict(cf_pd_test[my_lasso_lars_features]) - cf_pd_test["BOX"]) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % lr_model_small_lasso_lars.score(cf_pd_test[my_lasso_lars_features], cf_pd_test["BOX"]))
# Predict with all features
print('Predicted Box Office: %.2f' % lr_model_small_lasso_lars.predict(cf_in[my_lasso_lars_features]))

In [ ]:
from sklearn import linear_model
br_model = linear_model.BayesianRidge()
br_model.fit(cf_pd_train[my_lasso_lars_features], cf_pd_train["BOX"])


# The coefficients
print('Coefficients: \n', br_model.coef_)
# The mean square error
print("Residual sum of squares: %.2f"
      % np.mean((br_model.predict(cf_pd_test[my_lasso_lars_features]) - cf_pd_test["BOX"]) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % br_model.score(cf_pd_test[my_lasso_lars_features], cf_pd_test["BOX"]))
# Predict with all features
print('Predicted Box Office: %.2f' % br_model.predict(cf_in[my_lasso_lars_features]))

In [ ]:
from sklearn import linear_model
br_model = linear_model.BayesianRidge()
br_model.fit(cf_pd_train[my_features_lasso], cf_pd_train["BOX"])


# The coefficients
print('Coefficients: \n', br_model.coef_)
# The mean square error
print("Residual sum of squares: %.2f"
      % np.mean((br_model.predict(cf_pd_test[my_features_lasso]) - cf_pd_test["BOX"]) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % br_model.score(cf_pd_test[my_features_lasso], cf_pd_test["BOX"]))
# Predict with all features
print('Predicted Box Office: %.2f' % br_model.predict(cf_in[my_features_lasso]))


In [ ]:
rr_cv_model = linear_model.RidgeCV()
rr_cv_model.fit(cf_pd_train[my_features_lasso], cf_pd_train["BOX"])

# The alpha
print('alpha: ', rr_cv_model.alpha_)
# The coefficients
print('Coefficients: \n', rr_cv_model.coef_)
# The mean square error
print("Residual sum of squares: %.2f"
      % np.mean((rr_cv_model.predict(cf_pd_test[my_features_lasso]) - cf_pd_test["BOX"]) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % rr_cv_model.score(cf_pd_test[my_features_lasso], cf_pd_test["BOX"]))
# Predict with all features
print('Predicted Box Office: %.2f' % rr_cv_model.predict(cf_in[my_features_lasso]))

In [ ]:
rr_cv_model = linear_model.RidgeCV()
rr_cv_model.fit(cf_pd_train[my_lasso_lars_features], cf_pd_train["BOX"])

# The alpha
print('alpha: ', rr_cv_model.alpha_)
# The coefficients
print('Coefficients: \n', rr_cv_model.coef_)
# The mean square error
print("Residual sum of squares: %.2f"
      % np.mean((rr_cv_model.predict(cf_pd_test[my_lasso_lars_features]) - cf_pd_test["BOX"]) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % rr_cv_model.score(cf_pd_test[my_lasso_lars_features], cf_pd_test["BOX"]))
# Predict with all features
print('Predicted Box Office: %.2f' % rr_cv_model.predict(cf_in[my_lasso_lars_features]))